# Capacitor sourcing workbook

Use this workflow to build a robust standard capacitor library that is resilient to marketplace fluctuation.

In [39]:
%load_ext autoreload
import sys
import octopart
sys.path.append("../")
import scottopart.constants

octopart.logger.setLevel(20)  # disable annoying debug logs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
authorized_capacitor_manufacturers = ['TDK', 'Murata', 'Yageo', 'KEMET', 'Taiyo Yuden', 'Samsung', 'AVX']
authorized_sellers = ['Digi-Key', 'Newark', 'Mouser', 'Avnet', 'Arrow Electronics, Inc.']
authorized_country_codes = ['US']
authorized_packaging = ['Cut Tape', 'Tape & Reel', 'None']
query_package = ['0603']
query_capacitance = [4.7e-9]

for case_package in query_package:
    for capacitance in query_capacitance:
        print(case_package, capacitance)

0603 4.7e-09


In [49]:
# legacy search to replicate:
#    payload = {
#        'apikey': octopart_api_key,
#        'q': '', # leave blank
#        'filter[fields][manufacturer.displayname][]': authorized_capacitor_manufacturers,
#        'filter[fields][specs.capacitance.value][]': capacitance,
#        'filter[fields][specs.case_package.value][]': case_package,
#        'limit' : 100,
#        'include[]' : 'specs',
#    }


In [78]:
filter = {
    'manufacturer.displayname' : '(TDK Murata)',
    'specs.case_package.value' : '0603',
    'specs.capacitance.value'  : '4.7e-9'
}
r = octopart.search(query='', limit=100, filter_fields=filter, include_specs=True)

2018-08-25 14:34:57,081 - octopart.client - DEBUG: Requested Octopart URI: https://octopart.com/api/v3/parts/search?q=&start=0&limit=100&filter%5Bfields%5D%5Bmanufacturer.displayname%5D%5B%5D=%28TDK+Murata%29&filter%5Bfields%5D%5Bspecs.case_package.value%5D%5B%5D=0603&filter%5Bfields%5D%5Bspecs.capacitance.value%5D%5B%5D=4.7e-9&include%5B%5D=specs&apikey=c59ed28d


In [80]:
r

<PartsSearchResult: hits=0>

In [69]:
p.specs

{'capacitance': <Spec name=capacitance value=4.7e-09>,
 'capacitance_tolerance': <Spec name=capacitance_tolerance value=±5%>,
 'case_package': <Spec name=case_package value=['0603', '1608']>,
 'case_package_si': <Spec name=case_package_si value=1608>,
 'dielectric_characteristic': <Spec name=dielectric_characteristic value=C0G/NP0>,
 'dielectric_material': <Spec name=dielectric_material value=Ceramic Multilayer>,
 'lead_free_status': <Spec name=lead_free_status value=Lead Free>,
 'mounting_style': <Spec name=mounting_style value=Surface Mount>,
 'operating_temperature': <Spec name=operating_temperature value=[]>,
 'packaging': <Spec name=packaging value=['Tape & Reel (TR)', 'Cut Tape (CT)']>,
 'pin_count': <Spec name=pin_count value=2>,
 'reach_svhc_compliance': <Spec name=reach_svhc_compliance value=No SVHC>,
 'rohs_status': <Spec name=rohs_status value=Compliant>,
 'size_length': <Spec name=size_length value=0.0016>,
 'size_thickness': <Spec name=size_thickness value=0.0008>,
 'size_

In [70]:
from collections import defaultdict
filter_dict = defaultdict(list)

In [71]:
filter_dict = {
    'manufacturer.displayname' : 'TDK',
    'specs.case_package.value' : '0603',
    'specs.capacitance.value'  : '4.7e-9'
}

In [122]:
# We would do this if we can, but the octopart Python API
# won't accept a list for manufacturer.displayname.  So this
# is a work-around.
#filter = {
#    'manufacturer.displayname' : ['Murata', 'TDK', 'etc.']
#    'specs.case_package.value' : '0603',
#    'specs.capacitance.value'  : '4.7e-9'
#}
#r = octopart.search(query='', limit=100, filter_fields=filter, include_specs=True)

c = octopart.client.OctopartClient()
params = {
        'q': '', # leave blank
        'filter[fields][manufacturer.displayname][]': authorized_capacitor_manufacturers,
        'filter[fields][specs.capacitance.value][]': capacitance,
        'filter[fields][specs.case_package.value][]': case_package,
        'limit' : 100,
        'include[]' : 'specs',
}
r = octopart.models.PartsSearchResult(c._request('/parts/search', params=params))

In [123]:
r

<PartsSearchResult: hits=522>

In [105]:
from scottopart.mpnmagic.capacitors import parse_mpn

In [108]:
parsed_parts = [parse_mpn(p.mpn) for p in r.parts]

In [116]:
parsed_parts[10]

{'case_package': '0603',
 'voltage_rating_dc': 100,
 'dielectric_characteristic': 'X7R',
 'capacitance': 4.7e-09,
 'capacitance_tolerance': '10%',
 'reliability_code': 'A',
 'plating_code': 'Z',
 'packaging_code': '2',
 'special_code': 'A',
 'manufacturer': 'AVX'}

In [158]:
# filter the part offers
filtered_offers = [o for o in p.offers if o.seller in authorized_sellers and o.packaging in authorized_packaging]

In [160]:
authorized_in_stock_qty = sum([o.in_stock_quantity for o in filtered_offers])

In [161]:
authorized_in_stock_qty

106216

In [143]:
dir(p.offers[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_offer',
 'in_stock_quantity',
 'last_updated',
 'moq',
 'packaging',
 'prices',
 'product_url',
 'seller',
 'sku']

In [148]:
p.offers[3].moq

1

In [152]:
filtered_o[2].packaging

'Cut Tape'